# Experimenting with MAgent2 Environment

In [15]:
from magent2.environments import battle_v4
import numpy as np


In [1]:


env = battle_v4.env(map_size=20, render_mode='human', max_cycles=100000)
env.reset()
for agent in env.agent_iter():
    observation, reward, termination, truncation, info = env.last()
    if  termination or truncation:
        env.step(None)
    else:
        action = env.action_space(agent).sample()
        env.step(action)

    # If a team all terminated, break

env.close()

KeyboardInterrupt: 

In [38]:
env = battle_v4.env(map_size=45, render_mode="rgb_array", max_cycles=100000)
env.observation_spaces

from functools import reduce
env.observation_spaces['red_0'].shape
reduce((lambda x, y: x * y), env.observation_spaces['red_0'].shape)
# env.observation_space('red_0').sample().flatten().shape

845

In [46]:
env.reset()
env.action_spaces['red_0'].n
type (env)


pettingzoo.utils.wrappers.order_enforcing.OrderEnforcingWrapper

## Parallel API

In [1]:
from magent2.environments import battle_v4
env = battle_v4.parallel_env(map_size=20, render_mode='human', max_cycles=100000)
env.reset()

terminated_agents = set()
while env.agents:
    actions = {agent: env.action_space(agent).sample() for agent in env.agents}
    for agent in terminated_agents:
        actions[agent] = None

    observations, rewards, terminations, truncations, infos = env.step(actions)

    # Terminate agents that are done
    terminated_agents = {agent for agent, done in terminations.items() if done}
    truncated_agents = {agent for agent, done in truncations.items() if done}
    terminated_agents.update(truncated_agents)

env.close()

KeyboardInterrupt: 